# Kshitij Sonje :- CS24MTECH11025   
# Vineeth Chalsani :- CS24MTECH11023

In [3]:
import csv
import heapq
from collections import defaultdict

In [4]:
# Creates original direction connection in the flow network
def create_forward_connection(network_graph, connection_list, start_node, end_node, max_capacity, unit_cost):
    connection_id = len(connection_list)
    
    connection_list.append({
        'origin': start_node,
        'destination': end_node,
        'max_flow': max_capacity,
        'expense': unit_cost,
        'current_flow': 0,
        'opposite_id': connection_id + 1
    })
    
    network_graph[start_node].append(connection_id)
    return connection_id

# Creates reverse direction connection for residual network
def create_backward_connection(network_graph, connection_list, start_node, end_node, unit_cost):
    connection_id = len(connection_list)
    
    connection_list.append({
        'origin': end_node,
        'destination': start_node,
        'max_flow': 0,
        'expense': -unit_cost,
        'current_flow': 0,
        'opposite_id': connection_id - 1
    })
    
    network_graph[end_node].append(connection_id)
    return connection_id

# Adds bidirectional connection pair to the network
def insert_network_connection(network_graph, connection_list, start_node, end_node, max_capacity, unit_cost):
    create_forward_connection(network_graph, connection_list, start_node, end_node, max_capacity, unit_cost)
    create_backward_connection(network_graph, connection_list, start_node, end_node, unit_cost)

In [5]:
# Collects all unique vertices from connection network
def gather_network_vertices(connection_list):
    vertex_collection = set()
    link_index = 0
    while link_index < len(connection_list):
        link = connection_list[link_index]
        vertex_collection.add(link['origin'])
        vertex_collection.add(link['destination'])
        link_index += 1
    return vertex_collection

# Performs relaxation iterations for distance optimization
def execute_relaxation_cycles(connection_list, vertex_collection, distance_map, predecessor):
    cycle_counter = 0
    max_cycles = len(vertex_collection) - 1
    while cycle_counter < max_cycles:
        process_all_connections_for_relaxation(connection_list, distance_map, predecessor)
        cycle_counter += 1

# Processes each connection for distance relaxation
def process_all_connections_for_relaxation(connection_list, distance_map, predecessor):
    link_position = 0
    while link_position < len(connection_list):
        link = connection_list[link_position]
        attempt_path_improvement(link_position, link, distance_map, predecessor)
        link_position += 1

# Attempts to improve path distance through given connection
def attempt_path_improvement(link_position, link, distance_map, predecessor):
    start_vertex, finish_vertex = link['origin'], link['destination']
    remaining_capacity = link['max_flow'] - link['current_flow']
    travel_expense = link['expense']
    
    if remaining_capacity > 0 and distance_map[start_vertex] != float('inf'):
        if distance_map[start_vertex] + travel_expense < distance_map[finish_vertex]:
            distance_map[finish_vertex] = distance_map[start_vertex] + travel_expense
            predecessor[finish_vertex] = link_position

# Finds minimum cost path allowing negative edge weights
def shortest_path_with_negative_costs(connection_list, origin_vertex, target_vertex):
    distance_map = defaultdict(lambda: float('inf'))
    predecessor = {}
    distance_map[origin_vertex] = 0

    vertex_collection = gather_network_vertices(connection_list)
    execute_relaxation_cycles(connection_list, vertex_collection, distance_map, predecessor)

    if distance_map[target_vertex] == float('inf'):
        return None, None

    return distance_map, predecessor

In [6]:
# Processes all outgoing connections from current node
def examine_adjacent_connections(network_graph, connection_list, current_node, distance_tracker, route_tracker, node_potential, priority_heap):
    connection_position = 0
    adjacent_connections = network_graph[current_node]
    
    while connection_position < len(adjacent_connections):
        link_id = adjacent_connections[connection_position]
        link = connection_list[link_id]
        next_node = link['destination']
        spare_capacity = link['max_flow'] - link['current_flow']
        
        evaluate_connection_feasibility(link_id, link, current_node, next_node, spare_capacity, distance_tracker, route_tracker, node_potential, priority_heap)
        connection_position += 1

# Checks if connection has capacity and calculates reduced cost
def evaluate_connection_feasibility(link_id, link, current_node, next_node, spare_capacity, distance_tracker, route_tracker, node_potential, priority_heap):
    if spare_capacity > 0:
        modified_expense = link['expense'] + node_potential[current_node] - node_potential[next_node]
        attempt_distance_update(link_id, current_node, next_node, modified_expense, distance_tracker, route_tracker, priority_heap)

# Updates shortest distance if better path found
def attempt_distance_update(link_id, current_node, next_node, modified_expense, distance_tracker, route_tracker, priority_heap):
    if distance_tracker[current_node] + modified_expense < distance_tracker[next_node]:
        distance_tracker[next_node] = distance_tracker[current_node] + modified_expense
        route_tracker[next_node] = link_id
        heapq.heappush(priority_heap, (distance_tracker[next_node], next_node))

# Finds shortest path using modified costs based on dual variables
def optimized_shortest_path_search(network_graph, connection_list, origin_vertex, target_vertex, node_potential):
    distance_tracker = defaultdict(lambda: float('inf'))
    route_tracker = {}
    distance_tracker[origin_vertex] = 0

    priority_heap = [(0, origin_vertex)]
    processed_nodes = set()

    while priority_heap:
        current_distance, current_node = heapq.heappop(priority_heap)

        if current_node in processed_nodes:
            continue
        processed_nodes.add(current_node)

        if current_node == target_vertex:
            break

        examine_adjacent_connections(network_graph, connection_list, current_node, distance_tracker, route_tracker, node_potential, priority_heap)

    if distance_tracker[target_vertex] == float('inf'):
        return None, None

    return distance_tracker, route_tracker

In [7]:
# Displays algorithm header and configuration information
def display_algorithm_header(origin_vertex, target_vertex):
    separator_line = "=" * 70
    print(separator_line)
    print("MINIMUM COST MAXIMUM FLOW - NETWORK SIMPLEX METHOD")
    print(separator_line)
    print(f"Source vertex: {origin_vertex}")
    print(f"Sink vertex: {target_vertex}")
    print(separator_line)

# Reconstructs path from target back to origin using parent tracking
def reconstruct_route_from_target(connection_list, route_tracker, origin_vertex, target_vertex):
    route_sequence = []
    bottleneck_flow = float('inf')
    current_vertex = target_vertex
    
    step_counter = 0
    max_steps = len(route_tracker)
    while step_counter < max_steps:
        if current_vertex == origin_vertex:
            break
        
        link_index = route_tracker[current_vertex]
        link = connection_list[link_index]
        route_sequence.append(link_index)

        available_flow = link['max_flow'] - link['current_flow']
        bottleneck_flow = min(bottleneck_flow, available_flow)

        current_vertex = link['origin']
        step_counter += 1

    route_sequence.reverse()
    return route_sequence, bottleneck_flow

# Calculates total expense for given route
def compute_route_total_expense(connection_list, route_sequence):
    total_expense = 0
    route_position = 0
    while route_position < len(route_sequence):
        link_index = route_sequence[route_position]
        link = connection_list[link_index]
        total_expense += link['expense']
        route_position += 1
    return total_expense

# Updates flow values along the selected route
def update_flow_along_route(connection_list, route_sequence, flow_increment):
    route_position = 0
    while route_position < len(route_sequence):
        link_index = route_sequence[route_position]
        connection_list[link_index]['current_flow'] += flow_increment
        reverse_link_index = connection_list[link_index]['opposite_id']
        connection_list[reverse_link_index]['current_flow'] -= flow_increment
        route_position += 1

# Processes iteration results and updates algorithm state
def process_iteration_results(connection_list, route_sequence, bottleneck_flow, total_expense, accumulated_flow, accumulated_expense, cycle_number, distance_tracker, route_tracker, node_potential, origin_vertex, target_vertex, network_graph):
    cycle_expense = bottleneck_flow * total_expense
    accumulated_flow += bottleneck_flow
    accumulated_expense += cycle_expense

    print(f"Iteration {cycle_number}: FlowAdded={bottleneck_flow}, CostAdded={cycle_expense}, TotalCost={accumulated_expense}")

    potential_vertices = list(node_potential.keys()) + [origin_vertex, target_vertex]
    vertex_index = 0
    while vertex_index < len(potential_vertices):
        vertex = potential_vertices[vertex_index]
        if distance_tracker[vertex] != float('inf'):
            node_potential[vertex] += distance_tracker[vertex]
        vertex_index += 1

    distance_tracker, route_tracker = optimized_shortest_path_search(network_graph, connection_list, origin_vertex, target_vertex, node_potential)
    
    return accumulated_flow, accumulated_expense, distance_tracker, route_tracker

# Calculates optimal flow with minimum total expense using network simplex approach
def compute_optimal_minimum_cost_flow(network_graph, connection_list, origin_vertex, target_vertex):
    accumulated_flow = 0
    accumulated_expense = 0
    cycle_number = 0

    display_algorithm_header(origin_vertex, target_vertex)

    node_potential = defaultdict(int)
    distance_tracker, route_tracker = shortest_path_with_negative_costs(connection_list, origin_vertex, target_vertex)

    while route_tracker is not None:
        cycle_number += 1

        route_sequence, bottleneck_flow = reconstruct_route_from_target(connection_list, route_tracker, origin_vertex, target_vertex)
        total_expense = compute_route_total_expense(connection_list, route_sequence)
        update_flow_along_route(connection_list, route_sequence, bottleneck_flow)
        
        accumulated_flow, accumulated_expense, distance_tracker, route_tracker = process_iteration_results(
            connection_list, route_sequence, bottleneck_flow, total_expense, accumulated_flow, 
            accumulated_expense, cycle_number, distance_tracker, route_tracker, node_potential, 
            origin_vertex, target_vertex, network_graph
        )

    print()
    print("RESULT:")
    print(f"Requested flow: {accumulated_flow}")
    print(f"Achieved flow : {accumulated_flow}")
    print(f"Total min cost: {accumulated_expense}")

    return accumulated_flow, accumulated_expense

In [8]:
# Initializes network structures and reads CSV data
def initialize_network_and_load_data(filename):
    network_graph = defaultdict(list)
    connection_list = []

    with open(filename, 'r') as file_handle:
        data_rows = list(csv.reader(file_handle))
    
    if len(data_rows) != 4:
        raise ValueError(f"CSV must have exactly 4 rows, found {len(data_rows)}")
    
    return network_graph, connection_list, data_rows

# Processes all connections and adds them to network using while loop
def process_network_connections(network_graph, connection_list, origin_nodes, destination_nodes, flow_limits, connection_costs):
    total_connections = len(origin_nodes)
    connection_index = 0
    
    while connection_index < total_connections:
        insert_network_connection(network_graph, connection_list, origin_nodes[connection_index], 
                                destination_nodes[connection_index], flow_limits[connection_index], 
                                connection_costs[connection_index])
        connection_index += 1

# Reads network configuration from CSV file in assignment format
def load_network_from_csv(filename):
    network_graph, connection_list, data_rows = initialize_network_and_load_data(filename)

    origin_nodes = [int(x.strip()) for x in data_rows[0]]
    destination_nodes = [int(x.strip()) for x in data_rows[1]]
    flow_limits = [int(x.strip()) for x in data_rows[2]]
    connection_costs = [int(x.strip()) for x in data_rows[3]]

    total_connections = len(origin_nodes)
    total_vertices = len(set(origin_nodes + destination_nodes))

    print(f"Loaded graph with {total_connections} edges and {total_vertices} nodes.")
    print("Computing minimum cost maximum flow using Network Simplex...")

    process_network_connections(network_graph, connection_list, origin_nodes, destination_nodes, flow_limits, connection_costs)
    
    origin_vertex = 1
    target_vertex = 2
    return network_graph, connection_list, origin_vertex, target_vertex

In [9]:
# Read the test case from Testcase.csv
def main():
    """
    Main function to solve minimum cost maximum flow problem.
    Reads input from Testcase.csv and applies Network Simplex method.
    """
    try:
        network_graph, connection_list, origin_vertex, target_vertex = load_network_from_csv('Testcase.csv')
        accumulated_flow, accumulated_expense = compute_optimal_minimum_cost_flow(network_graph, connection_list, origin_vertex, target_vertex)
        
    except Exception as e:
        print(f"Error: {e}")

In [10]:
main()

Loaded graph with 3 edges and 4 nodes.
Computing minimum cost maximum flow using Network Simplex...
MINIMUM COST MAXIMUM FLOW - NETWORK SIMPLEX METHOD
Source vertex: 1
Sink vertex: 2
Iteration 1: FlowAdded=2, CostAdded=8, TotalCost=8

RESULT:
Requested flow: 2
Achieved flow : 2
Total min cost: 8
